## **LIBRERIAS**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

## **CARGANDO EL CONJUNTO DE DATOS**

In [ ]:
df_train = pd.read_excel("../Data/Melsol-test.xlsx")
df_train.head(5)

In [ ]:
df_train.info()

## **EDA (Analisis Exploratorio de datos)**

In [ ]:
df_train.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualización de la distribución de variables
sns.set(style="whitegrid")
plt.figure(figsize=(12, 8))

for i, column in enumerate(df_train.columns):
    plt.subplot(3, 3, i + 1)
    sns.histplot(df_train[column], kde=True)
    plt.title(f'Distribution of {column}')

plt.tight_layout()
plt.show()


## **VERIFICAR DATOS ATIPICOS**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar el estilo de los gráficos
sns.set(style="whitegrid")

# Seleccionar las columnas numéricas
numeric_columns = df_train.select_dtypes(include=['int64', 'float64']).columns

# Crear gráficos de caja para identificar valores atípicos
plt.figure(figsize=(15, 10))
for i, column in enumerate(numeric_columns):
    plt.subplot(3, 3, i + 1)
    sns.boxplot(x=df_train[column])
    plt.title(f'Boxplot of {column}')

plt.tight_layout()
plt.show()


**Observaciones**
* Columna PRODUCTOS ALMACENADOS
* Columna DEMNADA DEL PRODUCTO
* Columna PRODUCTOS VENDIDOS


In [ ]:
def handle_outliers(column, cap_value=None):
    # Calcula el rango intercuartílico (IQR)
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    
    # Define los límites superior e inferior para identificar los valores atípicos
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR
    
    # Acotar los valores atípicos
    if cap_value is not None:
        column = column.clip(lower=lower_limit, upper=cap_value)
    else:
        column = column.clip(lower=lower_limit, upper=upper_limit)
    
    return column

# Aplicar la función a las columnas con valores atípicos
df_train['PRODUCTOS ALMACENADOS'] = handle_outliers(df_train['PRODUCTOS ALMACENADOS'])
df_train['DEMANDA DEL PRODUCTO'] = handle_outliers(df_train['DEMANDA DEL PRODUCTO'])
df_train['PRODUCTOS VENDIDOS'] = handle_outliers(df_train['PRODUCTOS VENDIDOS'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de correlación
correlation_matrix = df_train.corr()

# Crear una máscara para la parte triangular inferior
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Configurar el estilo de los gráficos
sns.set(style="white")

# Crear la figura y el eje (axis)
plt.figure(figsize=(10, 8))

# Crear el mapa de calor con la matriz de correlación
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', fmt=".2f")

# Configurar el título
plt.title('Correlation Matrix - Upper Triangle Only')

# Mostrar el gráfico
plt.show()


In [ ]:
correlation_matrix

**OBSERVACIONES** 
*CORRELACION FUERTE POSITIVA*
* PRODUCTOS ALMACENADOS y DEMANDA DEL PRODUCTO: 0.838341
* PRODUCTOS ALMACENADOS y PRODUCTOS VENDIDOS: 0.848399
* DEMANDA DEL PRODUCTO y PRODUCTOS VENDIDOS: 0.978426
* Correlación Fuerte Negativa:

*CORRELACION FUERTE NEGATIVA*

* FESTIVIDAD y PRODUCTOS VENDIDOS: -0.237186

In [ ]:
import matplotlib.pyplot as plt

# Seleccionar solo la columna de correlaciones con "PRODUCTOS VENDIDOS"
correlations_with_target = correlation_matrix['PRODUCTOS VENDIDOS'].drop('PRODUCTOS VENDIDOS')

# Crear un gráfico de barras para visualizar las correlaciones
plt.figure(figsize=(10, 6))
correlations_with_target.sort_values().plot(kind='barh', color='skyblue')
plt.title('Correlación con "PRODUCTOS VENDIDOS"')
plt.xlabel('Correlación')
plt.ylabel('Variable')
plt.show()


## **PREPROCESAMIENTO**

In [ ]:
# Mostrar el número de valores únicos de cada columna
for column in df_train.columns:
    num_unique_values = df_train[column].nunique()
    print(f'Columna: {column}, Número de Valores Únicos: {num_unique_values}')

In [ ]:
import pandas as pd

def analizar_y_eliminar_ruido(df):
    """
    Analiza el número de valores únicos en cada columna y decide si eliminar la columna si la cantidad de valores únicos es igual a 1.

    Parámetros:
    - df: DataFrame de pandas

    Retorna:
    - DataFrame modificado sin las columnas identificadas como ruido.
    """

    # Inicializar una lista para almacenar las columnas a eliminar
    columnas_a_eliminar = []

    # Iterar sobre cada columna del DataFrame
    for columna in df.columns:
        # Verificar si la cantidad de valores únicos es igual a 1
        if df[columna].nunique() == 1:
            columnas_a_eliminar.append(columna)
            print(f'Columna "{columna}" tiene un solo valor único. Se considera ruido.')

    # Eliminar las columnas identificadas como ruido
    df_sin_ruido = df.drop(columnas_a_eliminar, axis=1)

    print(f'\nColumnas eliminadas: {columnas_a_eliminar}')

    return df_sin_ruido

# Ejemplo de uso
df_sin_ruido = analizar_y_eliminar_ruido(df_train)


In [ ]:
df_sin_ruido.columns

In [ ]:
df_sin_ruido.describe()

In [ ]:
df_sin_ruido.shape    

In [ ]:
# Definir las características (X) y la variable objetivo (y)
X = df_sin_ruido.drop('PRODUCTOS VENDIDOS', axis=1)
y = df_sin_ruido['PRODUCTOS VENDIDOS']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Imprimir las formas de los conjuntos resultantes
print("Forma de X_train:", X_train.shape)
print("Forma de X_test:", X_test.shape)
print("Forma de y_train:", y_train.shape)
print("Forma de y_test:", y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Crear el modelo de Random Forest Regression
bosque = RandomForestRegressor(n_estimators=100,
                                criterion="squared_error",
                                max_features="sqrt",
                                bootstrap=True,
                                oob_score=True,
                                random_state=42)

# Entrenar el modelo
bosque.fit(X_train.values, y_train.values)

# Predecir en un nuevo conjunto de datos
nuevos_datos = [[1, 10, 0.4, 5.0, 1, 0]]  # Ajusta estos valores según tus datos
prediccion = bosque.predict(nuevos_datos)
print("Predicción:", prediccion)

# Evaluar el modelo en el conjunto de prueba
predicciones_test = bosque.predict(X_test.values)
mse = mean_squared_error(y_test.values, predicciones_test)
print("Error cuadrático medio en el conjunto de prueba:", mse)

# Imprimir la puntuación R^2 en el conjunto de entrenamiento
print("Puntuación R^2 en el conjunto de entrenamiento:", bosque.score(X_train.values, y_train.values))

# Imprimir la puntuación R^2 en el conjunto de prueba
print("Puntuación R^2 en el conjunto de prueba:", bosque.score(X_test.values, y_test.values))

# Imprimir la puntuación "out-of-bag" (OOB)
print("Puntuación OOB:", bosque.oob_score_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer

# Crear el modelo de Random Forest Regression
bosque = RandomForestRegressor(n_estimators=100,
                                criterion="squared_error",  # Utilizar "squared_error" en lugar de "mse"
                                max_features="sqrt",
                                bootstrap=True,
                                oob_score=True,
                                random_state=42)

# Definir la métrica a utilizar (en este caso, negativo del Error Cuadrático Medio para que sea coherente con la validación cruzada)
metrica = make_scorer(mean_squared_error, greater_is_better=False)

# Realizar validación cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Puedes ajustar el número de divisiones (folds)
resultados_cross_val = cross_val_score(bosque, X.values, y.values, cv=kf, scoring=metrica)

# Imprimir los resultados de la validación cruzada
print("Resultados de la validación cruzada:")
print("MSE por fold:", resultados_cross_val)
print("Promedio MSE:", resultados_cross_val.mean())

# Entrenar el modelo en todo el conjunto de datos
bosque.fit(X.values, y.values)

# Predecir en un nuevo conjunto de datos
nuevos_datos = [[1, 10, 0.4, 5.0, 1,0]]  # Ajusta estos valores según tus datos
prediccion = bosque.predict(nuevos_datos)
print("Predicción:", prediccion)

# Imprimir la puntuación R^2 en el conjunto completo
print("Puntuación R^2 en el conjunto completo:", bosque.score(X.values, y.values))

# Imprimir la puntuación "out-of-bag" (OOB)
print("Puntuación OOB:", bosque.oob_score_)


In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

# Suponiendo que bosque es tu modelo entrenado
for i, arbol in enumerate(bosque.estimators_[:3]):  # Limitar a los primeros tres árboles
    plt.figure(figsize=(10, 7))
    tree.plot_tree(arbol, feature_names=X.columns.tolist(), filled=True, rounded=True)
    plt.title(f"Árbol {i+1}")
    plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_excel("Melsol-test.xlsx")

x = df.drop(["PRODUCTOS VENDIDOS"], axis=1)
y = df["PRODUCTOS VENDIDOS"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.40, random_state=42)
#sc = StandardScaler()
#x_train = sc.fit_transform(x_train)
#x_test = sc.transform(x_test)
model = RandomForestRegressor(random_state=42, n_estimators=300, max_depth=10)
regressor = model.fit(x_train, y_train)
y_pred = regressor.predict(x_test)

r2 = r2_score(y_test, y_pred)

print ("Mean Absolute Error:" , metrics.mean_absolute_error(y_test, y_pred))
print ("Mean Squared Error:" , metrics.mean_squared_error(y_test, y_pred))
print ("Root Mean Squared Error:" , np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print(f"R²: {r2}")

# Gráfico de Dispersión (Scatter Plot)
plt.figure(figsize = (10, 5))
plt.scatter (y_test, y_pred, color='red', label='Comparison of Prediction between Actual & Prediction data')
plt.legend()
plt.grid()
plt.title('Gráfico de Dispersión')
plt.xlabel('Predicciones')
plt.ylabel('Data Actual')
plt.show()

# Curva de regresión
x_range = np.linspace(min(y_test), max(y_test), 100)
y_range = x_range  # Línea de regresión ideal (y = x)

plt.scatter(y_test, y_pred)
plt.plot(x_range, y_range, color="red", linestyle="--", label="Línea de Regresión Ideal")
plt.xlabel("Predicciones")
plt.ylabel("Data Actual")
plt.title("Curva de Regresión")
plt.legend()
plt.show()

ventas_reales = pd.read_excel("Diclofenaco-prediccion.xlsx")

predicciones_nuevas = model.predict(ventas_reales)

print(f"Predicción de ventas para el siguiente mes 1: {predicciones_nuevas[0]}")
print(f"Predicción de ventas para el siguiente mes 2: {predicciones_nuevas[1]}")
print(f"Predicción de ventas para el siguiente mes 3: {predicciones_nuevas[2]}")
print(f"Predicción de ventas para el siguiente mes 4: {predicciones_nuevas[3]}")
print(f"Predicción de ventas para el siguiente mes 5: {predicciones_nuevas[4]}")
print(f"Predicción de ventas para el siguiente mes 6: {predicciones_nuevas[5]}")
print(f"Predicción de ventas para el siguiente mes 7: {predicciones_nuevas[6]}")
print(f"Predicción de ventas para el siguiente mes 8: {predicciones_nuevas[7]}")
print(f"Predicción de ventas para el siguiente mes 9: {predicciones_nuevas[8]}")
print(f"Predicción de ventas para el siguiente mes 10: {predicciones_nuevas[9]}")

In [ ]:
import pandas as pd
df = pd.read_excel("../Data/Data Set - Prueba-Test.xlsx", engine="openpyxl")

In [ ]:
df.shape